# 1. NumPy: broadcasting

**Broadcasting** - механизм при котором Numpy позволяет выполнять поэлементные операции между массивами разной формы, расширяя меньший по нужным осям

In [1]:
import numpy as np

In [2]:
a = np.array([1,2,3])
M = np.ones((3,3))
print(a)
print(M)
print(M+a)

[1 2 3]
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
[[2. 3. 4.]
 [2. 3. 4.]
 [2. 3. 4.]]


1. NumPy сравнивает размеры массивов справа налево
2. Если размеры совпадают или один из них равен 1 - они совместимы
3. Меньшая размерность расширяется виртуально без создания копий данных\
   **Применение:**
* Центрирование данных $X-X.mean(axis=0)$
* Применение операций со скалярами
* Быстрое линейное преобразование\
  **Важно!**
Если размерности не совместимы, то будет ошибка *ValueError*

# 2. Практика с broadcasting

## 2.1 Примеры использования

In [3]:
mat = np.random.randint(1,10,(4,5))
mean_per_col = mat.mean(axis=0)
centered = mat - mean_per_col
new_mean_per_col = centered.mean(axis=0)
print("Оригинал:\n", mat)
print("Центры колонок:\n", mean_per_col)
print("Центрированный массив:\n", centered)
print("Новые центры колонок:\n", new_mean_per_col)

Оригинал:
 [[6 3 2 9 2]
 [9 5 3 6 5]
 [5 1 9 3 1]
 [9 2 7 5 8]]
Центры колонок:
 [7.25 2.75 5.25 5.75 4.  ]
Центрированный массив:
 [[-1.25  0.25 -3.25  3.25 -2.  ]
 [ 1.75  2.25 -2.25  0.25  1.  ]
 [-2.25 -1.75  3.75 -2.75 -3.  ]
 [ 1.75 -0.75  1.75 -0.75  4.  ]]
Новые центры колонок:
 [0. 0. 0. 0. 0.]


In [4]:
row = np.array([10,20,30,40,50])
print(mat+row)

[[16 23 32 49 52]
 [19 25 33 46 55]
 [15 21 39 43 51]
 [19 22 37 45 58]]


# 3. Pandas: группировка (groupby), агрегация

**groupby** - метод, реализующий технику split-apply-combine
1. *split* - разделение на группы по значениям столбца
2. *apply* - применение функций, например sum(), mean(), count() и т.п.
3. *combine* - объединение результата в новый DataFrame

In [5]:
import pandas as pd

Определить среднее значение score на город:

In [6]:
df = pd.DataFrame({
    'city': ['A', 'B', 'A', 'B', 'C'],
    'score': [10, 20, 30, 40, 50],
    'age':    [20, 30, 40, 50, 60]
})
mean_scores = df.groupby('city')['score'].mean()
mean_scores

city
A    20.0
B    30.0
C    50.0
Name: score, dtype: float64

Применение нескольких агрегатов сразу

In [7]:
agg = df.groupby('city').agg({
    'score': ['mean','sum','count'],
    'age': 'mean'
})
agg

score             age
      mean sum count  mean
city                      
A     20.0  40     2  30.0
B     30.0  60     2  40.0
C     50.0  50     1  60.0

Группировка по двум столбцам

In [8]:
two = df.groupby(['city','age'])['score'].sum()
two

city  age
A     20     10
      40     30
B     30     20
      50     40
C     60     50
Name: score, dtype: int64

## 3.1 Практика с Groupby
Задача:
* Сгенерировать DataFrame с колонками city (из \['X','Y','Z'\], 200 строк), value(случайные числа 1-100), category(\['A','B'\])
* Посчитать среднее и сумму value в разрезе city, category
* Выполнить фильтрацию: оставить только группы city=Y, category = A

In [17]:
cities = ['X', 'Y', 'Z']
categories = ['A', 'B']
n=200
np.random.seed(42) #Для воспроизводимости
df2 = pd.DataFrame({
    'city': np.random.choice(cities, size=n),
    'value': np.random.randint(1, 101, size=n),
    'category': np.random.choice(categories, size=n)
})
df2.head()

,city,value,category
0,Z,52,A
1,X,62,B
2,Z,58,A
3,Z,52,B
4,X,12,A


In [19]:
aggregate_df = df2.groupby(['city','category'])['value'].agg(['mean','sum'])
aggregate_df

mean   sum
city category                 
X    A         34.869565   802
     B         51.837209  2229
Y    A         56.750000  1589
     B         49.636364  1638
Z    A         47.307692  1845
     B         53.852941  1831

In [31]:
aggregate_df.xs(('Y', 'A'), drop_level=True)

mean      56.75
sum     1589.00
Name: (Y, A), dtype: float64

aggregate_df.index — это **MultiIndex**, и в таком случае доступ по aggregate_df\["city"\] выдаёт ошибку, потому что city — не столбец, а уровень индекса.

**Столбец (column)**
Это часть самих данных DataFrame.
Доступ к столбцу — как к обычному атрибуту или через df\[col_name\].

**Уровень MultiIndex**
Это часть индекса, которая организует строки.
В MultiIndex один или несколько уровней индексируют строки, но не являются столбцами внутри данных.

Потому что Panda смотрит только в названия столбцов (aggregate_df.columns). В aggregate_df только \['mean', 'sum'\].\
Следовательно, aggregate_df\["city"\] — ошибка, потому что в именах столбцов такого нет.

In [29]:
aggregate_df.loc[('Y', 'A')]

mean      56.75
sum     1589.00
Name: (Y, A), dtype: float64

1. Возвращаемый тип и уровень индекса
    loc\[('Y','A')\]
    — использует точное соответствие полному индексу.\
    — Возвращает Series, но представление уровня индекса сохраняется.\
    — Если вставить кортеж в срез (loc\[('Y','A'), :\]), возвращается DataFrame.
    
    xs(('Y','A'))\
    — предназначен именно для MultiIndex, автоматически убирает уровень индекса, упрощая результат (drop_level=True по умолчанию)\

2. Поведение при drop_level\
    xs имеет аргумент drop_level, который по умолчанию — True.\
    Если указать drop_level=False, он сохранит оба уровня, возвращая DataFrame